In [94]:
import cv2
import numpy as np

import matplotlib.pyplot as plt

board_scores = [
        255, 250, 
         11,  12,  13, 
        181, 182, 183, 
         41,  42,  43, 
        131, 132, 133, 
         61,  62,  63, 
        101, 102, 103,
        151, 152, 153,
         21,  22,  23,
        171, 172, 173,
         31,  32,  33,
        191, 192, 193,
         71,  72,  73,
        161, 162, 163,
         81,  82,  83,
        111, 112, 113,
        141, 142, 143,
         91,  92,  93,
        121, 122, 123,
         51,  52,  53,
        201, 202, 203    
    ]

board_scores = [(0, score, 0) for score in board_scores]

In [95]:
def find_image_type(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_red = np.array([0, 100, 100])
    upper_red = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])

    mask_red1 = cv2.inRange(hsv_image, lower_red, upper_red)
    mask_red2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
    mask_red = mask_red1 + mask_red2
    mask_red = 1 * (mask_red > 0)
    red_pixel_diffs = []
    # Type1:
    num_red = mask_red[620:670,220:270].flatten().sum()
    p_type1 = num_red / mask_red[620:670,220:270].flatten().shape[0]
    red_pixel_diffs.append(p_type1)
    # Type2:
    num_red = mask_red[65:102, 641:679].flatten().sum()
    p_type2 = num_red / mask_red[65:102, 641:679].flatten().shape[0]
    red_pixel_diffs.append(p_type2)
    #Type3:mask[625:665, 634:673]
    num_red = mask_red[625:665, 634:673].flatten().sum()
    p_type3 = num_red / mask_red[625:665, 634:673].flatten().shape[0]
    red_pixel_diffs.append(p_type3)
    
    return 1 + np.argmax(np.array(red_pixel_diffs))

In [124]:
def get_first_last_frame_difference(video_path):
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)

    # Read the first frame
    ret, first_frame = video_capture.read()
    # Initialize variables to store the first and last frames
    last_frame = None

    # Loop through the video and read each frame until the last frame is reached
    while ret:
        ret, frame = video_capture.read()
        if ret:
            last_frame = frame

    # Release the video capture object
    video_capture.release()

    # Convert the frames to grayscale
    first_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
    last_gray = cv2.cvtColor(last_frame, cv2.COLOR_BGR2GRAY)

    frame_difference = cv2.absdiff(first_gray, last_gray)
    _, mask = cv2.threshold(frame_difference, 50, 255, cv2.THRESH_BINARY)
    # mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel=np.ones((7, 7), dtype=np.uint8))

    return first_frame, last_frame, mask


In [181]:
def decode_score(score):
    if score == 255:
        return "b50"
    if score == 250:
        return "b25" 

    multipliers = {1: "s", 2: "d", 3: "t"}
    multiplier = multipliers[score % 10]
    return f"{multiplier}{score//10}"

def find_location(tip, mask):
    if len(mask.shape) > 2:
        mask = mask[:, :, 1]
    margin = 4
    y, x = tip
    
    sample = mask[x - margin : x + margin, y - margin : y + margin].flatten()
    unique_elements, element_counts = np.unique(sample, return_counts=True)

    most_common_index = np.argmax(element_counts)
    score =  unique_elements[most_common_index]
    return decode_score(score)

def find_leftmost_point(contour):
    leftmost_point = None
    contour = contour.squeeze()
    min_x = np.min(contour[:, 0])
    if leftmost_point is None or min_x < leftmost_point[0]:
        leftmost_point = (min_x, contour[contour[:, 0].argmin(), 1])
    return leftmost_point


def solve_type_1(frame_difference):
    img = frame_difference.copy()
    mask = cv2.imread("task3_type1mask.png")    
    segments = np.isin(mask, board_scores).all(axis=-1)
    img[~segments] = 0
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contours = np.concatenate(contours, axis=0)
    tip = find_leftmost_point(contours)
    score = find_location(tip, mask)
    print(score)

def solve_type_2(frame_difference):
    img = frame_difference.copy()
    mask = cv2.imread("task3_type2mask.png")    
    segments = np.isin(mask, board_scores).all(axis=-1)
    img[~segments] = 0
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contours = np.concatenate(contours, axis=0)
    tip = find_leftmost_point(contours)
    score = find_location(tip, mask)
    print(score)


def solve_type_3(frame_difference):
    img = frame_difference.copy()
    mask = cv2.imread("task3_type3mask.png")    
    segments = np.isin(mask, board_scores).all(axis=-1)
    img[~segments] = 0
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contours = np.concatenate(contours, axis=0)
    tip = find_leftmost_point(contours)
    score = find_location(tip, mask)
    print(score)


def find_score(frame_difference, type):
    img = frame_difference.copy()
    mask = cv2.imread(f"task3_type{type}mask.png")    
    segments = np.isin(mask, board_scores).all(axis=-1)
    img[~segments] = 0
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    contours = np.concatenate(contours, axis=0)
    tip = find_leftmost_point(contours)
    score = find_location(tip, mask)
    print(score)



for i in range(1,26):
    path = f'train/Task3/{ "0" + str(i) if i<10 else i}.mp4'
    first_frame, last_frame, frame_difference = get_first_last_frame_difference(path)

    
    type = find_image_type(first_frame)



    print(path, type)
    score = find_score(frame_difference, type)

train/Task3/01.mp4 1
s1
train/Task3/02.mp4 2
s19
train/Task3/03.mp4 2
t19
train/Task3/04.mp4 1
s20
train/Task3/05.mp4 3
s20
train/Task3/06.mp4 1
s5
train/Task3/07.mp4 2
s7
train/Task3/08.mp4 3
t20
train/Task3/09.mp4 1
t5
train/Task3/10.mp4 2
s17
train/Task3/11.mp4 3
t14
train/Task3/12.mp4 3
s18
train/Task3/13.mp4 1
s18
train/Task3/14.mp4 2
s17
train/Task3/15.mp4 3
t20
train/Task3/16.mp4 3
b25
train/Task3/17.mp4 1
t1
train/Task3/18.mp4 2
s3
train/Task3/19.mp4 1
s5
train/Task3/20.mp4 3
t20
train/Task3/21.mp4 2
t7
train/Task3/22.mp4 1
s20
train/Task3/23.mp4 1
s5
train/Task3/24.mp4 3
s14
train/Task3/25.mp4 1
s5


In [ ]:
board_scores = [
        255, 250, 
         11,  12,  13, 
        181, 182, 183, 
         41,  42,  43, 
        131, 132, 133, 
         61,  62,  63, 
        101, 102, 103,
        151, 152, 153,
         21,  22,  23,
        171, 172, 173,
         31,  32,  33,
        191, 192, 193,
         71,  72,  73,
        161, 162, 163,
         81,  82,  83,
        111, 112, 113,
        141, 142, 143,
         91,  92,  93,
        121, 122, 123,
         51,  52,  53,
        201, 202, 203    
    ]

board_scores = [(0, score, 0) for score in board_scores]
mask = cv2.imread("task3_type1mask.png")

segments = np.isin(mask, board_scores)



# segments = (mask[segments][:, 0] == 0)
# segments = (mask[segments][:, 2] == 0)



In [170]:
def decode_score(score):
    
    if score == 255:
        return "b50"
    
    if score == 250:
        return "b25" 

    multipliers = {1: "s", 2: "d", 3: "t"}
    multiplier = multipliers[score % 10]
    return f"{multiplier}{score//10}"

for i in board_scores:
    print(f"{i} => {decode_score(i[1])}")


(0, 255, 0) => b50
(0, 250, 0) => b25
(0, 11, 0) => s1
(0, 12, 0) => d1
(0, 13, 0) => t1
(0, 181, 0) => s18
(0, 182, 0) => d18
(0, 183, 0) => t18
(0, 41, 0) => s4
(0, 42, 0) => d4
(0, 43, 0) => t4
(0, 131, 0) => s13
(0, 132, 0) => d13
(0, 133, 0) => t13
(0, 61, 0) => s6
(0, 62, 0) => d6
(0, 63, 0) => t6
(0, 101, 0) => s10
(0, 102, 0) => d10
(0, 103, 0) => t10
(0, 151, 0) => s15
(0, 152, 0) => d15
(0, 153, 0) => t15
(0, 21, 0) => s2
(0, 22, 0) => d2
(0, 23, 0) => t2
(0, 171, 0) => s17
(0, 172, 0) => d17
(0, 173, 0) => t17
(0, 31, 0) => s3
(0, 32, 0) => d3
(0, 33, 0) => t3
(0, 191, 0) => s19
(0, 192, 0) => d19
(0, 193, 0) => t19
(0, 71, 0) => s7
(0, 72, 0) => d7
(0, 73, 0) => t7
(0, 161, 0) => s16
(0, 162, 0) => d16
(0, 163, 0) => t16
(0, 81, 0) => s8
(0, 82, 0) => d8
(0, 83, 0) => t8
(0, 111, 0) => s11
(0, 112, 0) => d11
(0, 113, 0) => t11
(0, 141, 0) => s14
(0, 142, 0) => d14
(0, 143, 0) => t14
(0, 91, 0) => s9
(0, 92, 0) => d9
(0, 93, 0) => t9
(0, 121, 0) => s12
(0, 122, 0) => d12
(0,

In [73]:
def find_image_type(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_red = np.array([0, 100, 100])
    upper_red = np.array([10, 255, 255])
    lower_red2 = np.array([160, 100, 100])
    upper_red2 = np.array([180, 255, 255])

    mask_red1 = cv2.inRange(hsv_image, lower_red, upper_red)
    mask_red2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
    mask_red = mask_red1 + mask_red2
    mask_red = 1 * (mask_red > 0)

    # Type1:
    num_red = mask_red[620:670,220:270].flatten().sum()
    p_type1 = num_red / mask_red[620:670,220:270].flatten().shape[0]

    # Type2:
    num_red = mask_red[65:102, 641:679].flatten().sum()
    p_type2 = num_red / mask_red[65:102, 641:679].flatten().shape[0]

    #Type3:mask[625:665, 634:673]
    num_red = mask_red[625:665, 634:673].flatten().sum()
    p_type3 = num_red / mask_red[625:665, 634:673].flatten().shape[0]

    print(f"Type1: {p_type1}  Type2: {p_type2}  Type3: {p_type3}")

find_image_type(first_frame)

Type1: 0.0  Type2: 0.0  Type3: 0.6474358974358975
